In [1]:
!pip install numba

     ---------------------------------------- 2.6/2.6 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 15.6/15.6 MB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 28.1/28.1 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1


In [4]:
import numpy as np
from numba import cuda

# Define the matrix multiplication kernel
@cuda.jit
def matrix_multiplication(A, B, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

# Main function for matrix multiplication on GPU
def matrix_multiply_on_gpu(A, B):
    # Allocate memory on the GPU
    A_global_mem = cuda.to_device(A)
    B_global_mem = cuda.to_device(B)
    C_global_mem = cuda.device_array((A.shape[0], B.shape[1]))

    # Configure the grid and block dimensions
    threads_per_block = (16, 16)
    blocks_per_grid_x = int(np.ceil(A.shape[0] / threads_per_block[0]))
    blocks_per_grid_y = int(np.ceil(B.shape[1] / threads_per_block[1]))
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    # Launch the kernel
    matrix_multiplication[blocks_per_grid, threads_per_block](A_global_mem, B_global_mem, C_global_mem)

    # Copy the result back to the host
    C = C_global_mem.copy_to_host()

    return C

# Example usage
if __name__ == "__main__":
    # Define matrices A and B
    A = np.random.rand(100, 100)
    B = np.random.rand(100, 100)

    # Perform matrix multiplication on GPU
    result = matrix_multiply_on_gpu(A, B)

    # Perform matrix multiplication on CPU for comparison
    cpu_result = np.dot(A, B)

    # Check if the results from GPU and CPU are the same
    print("Results are equal: ", np.allclose(result, cpu_result))


c:\Users\prajw\anaconda3\envs\DeepLearning\lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 49 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\prajw\anaconda3\envs\DeepLearning\lib\site-packages\numba\core\utils.py:213: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-captured-errors
Exception origin:
  File "c:\Users\prajw\anaconda3\envs\DeepLearning\lib\site-packages\numba\cuda\cudadrv\nvvm.py", line 144, in __new__
    raise NvvmSupportError(errmsg % e)

  warnings.warn(msg,


NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
Could not find module 'nvvm.dll' (or one of its dependencies). Try using the full path with constructor syntax.